# Overview

The below script will extract data from multiple sources and file types (csv, json, xml) before performing a simple cleaning tasks and finally outputting a single csv file. The csv output file will be a collation of all the data ingested, post the simple cleaning transformation. <br>
Of course, this was just a fun evening project and the code has not been designed for any kind of commercial application.
<br><br>
The data is a very simple dataset comprising of a handful of people and a log of their names, height and weight. In the data file holding each of the datasets exists a three duplicates of the same data. Again, the data itself is not important, more so that I had a simple collection of multiple sources of data to play around with. 
<br><br>
I have also taken a very 'functional programming' approach here where all of the operations have been broken down into smaller modularised functions that will then be called within other functions representing the extract, transform and load operations of an automated ETL pipeline. This can (or will be) further generalised into a final single function that will take an arguement of one file path, directing the script to the directory containing the many differing sources of data. This final function will then output a single collation of these data sources as one .csv file and an accompanying etl_process_log.text file containing the details of each stage of the etl process and that stage's corresponding timestamp.
<br><br>
For the sake of true automation, the below code can easily be copied into a script text file and executed via an OS's automated task command.

# Set Up

In [1]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime

# Building The Functions

### Extract Functions

In [2]:
# Function to read in a csv file
'''
This function will read in a csv file and output a pandas dataframe object
Input: file path
Output: pandas object
Params: file_to_process
'''
def extract_from_csv(file_to_process): 
    col_names = ['Name', 'Height', 'Weight']
    dataframe = pd.read_csv(file_to_process) 
    dataframe.columns = col_names
    return dataframe





# Function to read in multiple csv files
'''
This function will read in a list of csv files and output a pandas dataframe object
Input: iterable list of file paths
Output: pandas object
Params: path_list
'''
def multiple_csv(path_list):
    df_list = []
    for path in path_list:
        dataframe = extract_from_csv(path)
        df_list.append(dataframe)
        
    return pd.concat(df_list, ignore_index=True)


In [3]:
# Function to read in a json file
'''
This function will read in a csv file and output a pandas dataframe object
Input: file path
Output: pandas object
Params: file_to_process
'''
def extract_from_json(file_to_process): 
    col_names = ['Name', 'Height', 'Weight']
    dataframe = pd.read_json(file_to_process, lines=True) 
    dataframe.columns = col_names
    return dataframe 





# Function to read in multiple json files
'''
This function will read in a list of json files and output a pandas dataframe object
Input: iterable list of file paths
Output: pandas object
Params: path_list
'''
def multiple_json(path_list):
    df_list = []
    for path in path_list:
        dataframe = extract_from_json(path)
        df_list.append(dataframe)
        
    return pd.concat(df_list, ignore_index=True)

In [4]:
# Function to read in an xml file
'''
This function will read in a csv file and output a pandas dataframe object. This assumes a very specific xml tree schema and is not broadly
applicable to any other xml schemas. 
Input: file path
Output: pandas object
Params: file_to_process
'''
def extract_from_xml(file_to_process):
    # Create an empty dataframe 
    col_names = ['Name', 'Height', 'Weight']
    dataframe = pd.DataFrame(columns = col_names)
    
    # Create tree object and acess the root element
    tree = ET.parse(file_to_process)
    root_element = tree.getroot()
    
    # Iterate through each child of the root element (in this case this is an individual person)
    for child in root_element:
        # Accessing name, height and weight of each person
        name = child.find('name').text
        height = child.find('height').text
        weight = child.find('weight').text
        
        # Appending new row of data to existing dataframe
        dataframe.loc[len(dataframe), :] = [name, height, weight]
    
    return dataframe
    
    



# Function to read in multiple xml files
'''
This function will read in a list of xml files and output a pandas dataframe object
Input: iterable list of file paths
Output: pandas object
Params: path_list
'''
def multiple_xml(path_list):
    df_list = []
    for path in path_list:
        dataframe = extract_from_xml(path)
        df_list.append(dataframe)
        
    return pd.concat(df_list, ignore_index=True)

In [8]:
# Function to batch ingest multiple files
'''
This function will take a file path pointing towards a directory (folder) and return a single dataframe object containing a collation of all the
data found in that directory, assuming that the file types containing such data are either .csv, .xml or .json
Input: directory file path
Output: pandas dataframe
Params: file_path
'''
def batch_extract(file_path):
    # Creating a list of file paths found in the given directory. One list for each file type
    csv_paths = glob.glob(file_path + '*.csv')
    json_paths = glob.glob(file_path + '*.json')
    xml_paths = glob.glob(file_path + '*.xml')
    
    # Extract all csv files and return a dataframe
    csv_df = multiple_csv(csv_paths)
    
    # Extract all json files and return a dataframe
    json_df = multiple_json(json_paths)
    
    # Extract all xml files and return a dataframe
    xml_df = multiple_xml(xml_paths)
    
    
    return pd.concat([csv_df, json_df, xml_df],ignore_index=True)
    
    
    
    

In [11]:
batch_extract('data/').sample(10)

,Name,Height,Weight
35,simon,67.90,112.37
38,ivan,67.62,114.14
14,joe,67.79,144.3
16,tom,69.8,141.49
3,ravi,68.22,142.34
34,ivan,67.62,114.14
1,ajay,71.52,136.49
7,alice,69.4,153.03
5,alex,65.78,112.99
13,ravi,68.22,142.34


### Transform Functions

### Load Functions

### Process Log Functions

# Final Script